In [1]:
import pandas as pd
import os
import re
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

# Create initial datasets

### Setup input/output

In [2]:
# set base directory
#baseDIR = '/mnt/d'
baseDIR = 'T:/'

# set input directory
#inDIR = 'TBNG/data/veg/PVSAGE'
inDIR = "2-Projects/Thunder Basin/PrecipVariability/Data Entry/Multi-year Data/herbaceous/old"

# set input file
#inXLSX = 'PVSAGEmulti-year_Herb_9November2021SD.xlsx'
inXLSX = 'PVSAGEmulti-year_Herb_19Jan2022CE_checking.xlsx'

### Load data

In [3]:
# create an Excel file object for investigating contents
#pvs_xlsx = pd.ExcelFile(os.path.join(baseDIR, inDIR, inXLSX))
pvs_xlsx = pd.ExcelFile(os.path.join('data/', inXLSX))

# display the sheet names in the .xlsx file
display(pvs_xlsx.sheet_names)

['metadata',
 'sentek tube installation notes',
 'sentek notes',
 'grass & forb heights',
 'basal cover',
 'phenology',
 'aerial cover',
 'herbaceous density',
 'forage quality',
 '2017 FQ baseline fxnal group',
 'baseline fxnal group FQ 2017',
 'pin frame',
 'pin frame calibration',
 'pinframe calibration clipping',
 'light reading',
 'light reading times']

In [4]:
# Create the desired dataframes by parsing the individual sheets
df_pvs_bm = pvs_xlsx.parse(sheet_name='pinframe calibration clipping', header=0)
df_pvs_pf = pvs_xlsx.parse(sheet_name='pin frame calibration', header=0)

# Biomass dataset

#### Clean up dates

In [5]:
# create a mask to identify all values that are 'date-like'
dates_bm_mask = df_pvs_bm['Date'].map(lambda x: bool(re.match('\d\d\d\d-\d\d-\d\d 00:00:00', str(x))))
# summarize to find all unique values in the 'Date' column for each year
dates_yrly_bm = pd.DataFrame(df_pvs_bm.groupby('Year')['Date'].unique()).explode('Date')
# create a mask of 'date-like' values from the unique values pulled for each year
dates_yrly_bm_mask = dates_yrly_bm['Date'].map(lambda x: bool(re.match('\d\d\d\d-\d\d-\d\d 00:00:00', str(x))))
# create a Series of the 'date-like' unique values
dates_yrly_bm_good = dates_yrly_bm['Date'][dates_yrly_bm_mask].astype(np.datetime64)
# get the average date for each month of each year for manually correcting vague dates
dates_yrly_bm_avgs = dates_yrly_bm_good.groupby(['Year', dates_yrly_bm_good.dt.month_name()]).mean().dt.date

In [6]:
# display the average dates for each month of each year
display(dates_yrly_bm_avgs)

Year  Date
2017  July    2017-07-12
2019  July    2019-07-23
2020  July    2020-07-26
2021  July    2021-07-26
Name: Date, dtype: object

In [7]:
# display the unique non-'date-like' values present in each year
display(dates_yrly_bm['Date'][~dates_yrly_bm_mask])

Year
2018    7/23/18 to 7/25/18
Name: Date, dtype: object

In [8]:
# create a dictionary to manually change non-'date-like' values to dates
dates_bm_dict = {
    2018: {
        '7/23/18 to 7/25/18': pd.to_datetime('2018-07-24').date()
    }
}

In [9]:
# Create a new column of cleaned up dates from 'date-like' values and manual replacement
df_pvs_bm.loc[dates_bm_mask, 
              'Date_cln'] = df_pvs_bm['Date'][dates_bm_mask].astype(np.datetime64).dt.date
df_pvs_bm.loc[~dates_bm_mask, 
              'Date_cln'] = df_pvs_bm[~dates_bm_mask].apply(lambda x: dates_bm_dict[x.Year][x.Date], axis=1)
df_pvs_bm['Date_cln'] = df_pvs_bm['Date_cln'].astype(np.datetime64)



### Prep biomass values for joining to pin frame

In [10]:
# remove all rows with NA values for Total weight signified with negative values
df_pvs_bm = df_pvs_bm[df_pvs_bm['total weight'] >= 0]

# drop any records missing Biomass data
df_pvs_bm = df_pvs_bm[df_pvs_bm['total weight'].notnull()]

In [11]:
# fill all functional group clipping NA values with zeros for averaging
df_pvs_bm.loc[
    :, df_pvs_bm.columns.str.contains('eight$')] = df_pvs_bm.loc[
    :, df_pvs_bm.columns.str.contains('eight$')].fillna(0)

# remove all rows with negative values or convert to NAN
remove_indv_na = False
if remove_indv_na:
    df_pvs_bm = df_pvs_bm[~(
        df_pvs_bm.loc[:, df_pvs_bm.columns.str.contains('eight$')] < 0).any(axis=1)]
else:
    for c in df_pvs_bm.columns[df_pvs_bm.columns.str.contains('eight$')]:
        df_pvs_bm[c] = df_pvs_bm[c].apply(lambda x: np.nan if x < 0 else x)

In [12]:
# drop 'loc' from plot data for consistency
df_pvs_bm['Plot Name'] = df_pvs_bm['Plot Name'].astype('str').apply(lambda x: int(re.sub('loc', '', x)))

In [13]:
# create additional functional group groupings
df_pvs_bm['annual grass weight'] = df_pvs_bm['brome weight'] + df_pvs_bm['VUOC weight']

In [14]:
df_pvs_bm[df_pvs_bm['Year'] == 2019][['forb weight', 'AF weight', 'PF weight']]

forb weight  AF weight  PF weight
36          0.0      20.10       0.35
37          0.0      28.70       0.00
38          0.0      21.35       0.00
39          0.0      12.40       0.00
40          0.0      22.10       0.35
41          0.0      13.85       0.55
42          0.0       1.00       2.10
43          0.0       3.10       0.00
44          0.0       1.35       0.00
45          0.0       1.20       1.60
46          0.0       0.35       2.60
47          0.0       0.25       0.40

In [15]:
# create consistent Forbs weight across all years
forb_years = df_pvs_bm.loc[((df_pvs_bm['forb weight'].isnull()) | (df_pvs_bm['forb weight'] == 0)) & 
              (df_pvs_bm[['AF weight', 'PF weight']].sum(axis=1) != 0),
                           'Year'].unique()
df_pvs_bm['forb weight'] = df_pvs_bm.apply(
    lambda x: x['AF weight'] + x['PF weight'] if x['Year'] in forb_years else x['forb weight'],
    axis=1)

# create consistent C4 weight across all years
c4_years = df_pvs_bm['Year'].unique()[(df_pvs_bm.groupby(
    'Year')[['C4 weight', 'BOBU weight']]).apply(
    lambda x: ~x['BOBU weight'].isnull().all() & ~x['C4 weight'].isnull().all())]

df_pvs_bm['C4 weight'] = df_pvs_bm.apply(
    lambda x: x['C4 weight'] + x['BOBU weight'] if x['Year'] in c4_years else x['C4 weight'],
    axis=1)

In [16]:
# recalculate areal density for all FG's
for c in df_pvs_bm.columns[df_pvs_bm.columns.str.contains('eight$')]:
        df_pvs_bm[c + ' g/m^2'] = df_pvs_bm[c] * 2.0

In [17]:
forb_years

array([2019, 2020, 2021], dtype=int64)

In [18]:
df_pvs_bm.loc[((df_pvs_bm['forb weight'].isnull()) | (df_pvs_bm['forb weight'] == 0)) & 
              (df_pvs_bm[['AF weight', 'PF weight']].sum(axis=1) != 0)]

Empty DataFrame
Columns: [Entered By, Observers, Recorder, Year, Date, Collection, Site, Plot Name, GPS N, GPS E, Brome, C3P, C4P, Other grass, Annual/Biennial Forb, Perennial Forb, Forb (2017-2018), Sub-shrub, VUOC, Standing dead, brome weight, C3 weight, C4 weight, BOBU weight, Other C4 weight, forb weight, AF weight, PF weight, VUOC weight, SD weight, total weight, brome weight g/m^2, C3 weight g/m^2, C4 weight g/m^2, forb weight g/m^2, AF weight g/m^2, PF weight g/m^2, SD weight g/m^2, VUOC weight g/m^2, BOBU weight g/m^2, Other C4 weight g/m^2, total weight g/m^2, % shrub cover, OPPO Cladodes, OPPO seedlings, TOTAL OPPO cladodes, notes, Date_cln, annual grass weight, annual grass weight g/m^2]
Index: []

[0 rows x 50 columns]

In [19]:
df_pvs_bm.loc[((df_pvs_bm['C4 weight g/m^2'].isnull()) | (df_pvs_bm['C4 weight g/m^2'] == 0)) & 
              (df_pvs_bm['BOBU weight g/m^2'] != 0)]

Empty DataFrame
Columns: [Entered By, Observers, Recorder, Year, Date, Collection, Site, Plot Name, GPS N, GPS E, Brome, C3P, C4P, Other grass, Annual/Biennial Forb, Perennial Forb, Forb (2017-2018), Sub-shrub, VUOC, Standing dead, brome weight, C3 weight, C4 weight, BOBU weight, Other C4 weight, forb weight, AF weight, PF weight, VUOC weight, SD weight, total weight, brome weight g/m^2, C3 weight g/m^2, C4 weight g/m^2, forb weight g/m^2, AF weight g/m^2, PF weight g/m^2, SD weight g/m^2, VUOC weight g/m^2, BOBU weight g/m^2, Other C4 weight g/m^2, total weight g/m^2, % shrub cover, OPPO Cladodes, OPPO seedlings, TOTAL OPPO cladodes, notes, Date_cln, annual grass weight, annual grass weight g/m^2]
Index: []

[0 rows x 50 columns]

### Visualize data to check that it makes sense

In [20]:
# plot pin-frame calibration data
sns.relplot(x='Date_cln', y='total weight g/m^2', hue='Site',
            data=df_pvs_bm, kind='line', marker="o")

# Pin Frame Dataset

### Combine Biomass and Pin Frame datasets

In [21]:
# drop 'loc' from plot data for consistency
df_pvs_pf['Plot Name'] = df_pvs_pf['Plot Name'].astype('str').apply(lambda x: int(re.sub('loc', '', x)))

# Create a new column in the Pin Frame dataframe pulling the nearest Biomass sampling date
df_pvs_pf = pd.merge_asof(df_pvs_pf.sort_values('Date'), 
                          df_pvs_bm.groupby(['Date_cln', 'Site', 'Plot Name']).mean().reset_index().sort_values('Date_cln')[['Site',
                                                                                                                             'Plot Name',
                                                                                                                             'C4 weight g/m^2',
                                                                                                                             'C3 weight g/m^2', 
                                                                                                                             'forb weight g/m^2', 
                                                                                                                             'brome weight g/m^2',
                                                                                                                             'VUOC weight g/m^2',
                                                                                                                             'annual grass weight g/m^2', 
                                                                                                                             'SD weight g/m^2',
                                                                                                                             'total weight g/m^2',
                                                                                                                             'Date_cln']],
                          left_on='Date',
                          right_on='Date_cln',
                          by=['Site', 'Plot Name'],
                          direction='nearest',
                          tolerance=datetime.timedelta(days=15),
                          suffixes=['_x', ''])

In [22]:
# drop any records missing Biomass data
df_pvs_pf = df_pvs_pf[df_pvs_pf['total weight g/m^2'].notnull()]

### Clean up Pin Frame dataset

In [23]:
# drop basal hits
df_pvs_pf = df_pvs_pf[df_pvs_pf['Position'] != 'basal']

In [24]:
# calculate forb hits
df_pvs_pf['Forb'] = df_pvs_pf['AnnualForb'] + df_pvs_pf['PerennialForb']

# calculate annual grass hits
df_pvs_pf['Annual Grass'] = df_pvs_pf['Brome'] + df_pvs_pf['VUOC']

### Calculate appropriate values at the experimental unit level

In [25]:
# aggregate data by experimental unit
df_pvs_pf_agg = df_pvs_pf.groupby(
    ['Site', 'Plot Name', 'Date_cln']).aggregate(
    {'Total Hits': np.sum,
     'C4P_Grass': np.sum,
     'C3P_Grass': np.sum,
     'Annual Grass': np.sum,
     'Forb': np.sum,
     'Brome': np.sum,
     'VUOC': np.sum,
     'StandingDead': np.sum,
     'TotalHerbHits': np.sum,
     'total weight g/m^2': np.mean,
     'C4 weight g/m^2': np.mean,
     'C3 weight g/m^2': np.mean,
     'forb weight g/m^2': np.mean,
     'brome weight g/m^2': np.mean,
     'VUOC weight g/m^2': np.mean,
     'annual grass weight g/m^2': np.mean,
     'SD weight g/m^2': np.mean}).reset_index()

In [26]:
pf_colname_mapper = {
    'C4P_Grass': 'hits_C4',
    'C3P_Grass': 'hits_C3',
    'Annual Grass': 'hits_AnnGrass',
    'Forb': 'hits_Forb',
    'Brome': 'hits_Brome',
    'VUOC': 'hits_VUOC',
    'StandingDead': 'hits_SD',
    'TotalHerbHits': 'hits_Total',
    'C4 weight g/m^2': 'weight_C4',
    'C3 weight g/m^2': 'weight_C3',
    'annual grass weight g/m^2': 'weight_AnnGrass',
    'forb weight g/m^2': 'weight_Forb',
    'brome weight g/m^2': 'weight_Brome',
    'VUOC weight g/m^2': 'weight_VUOC',
    'SD weight g/m^2': 'weight_SD',
    'total weight g/m^2': 'weight_Total'
}

df_pvs_pf_agg = df_pvs_pf_agg.rename(columns=pf_colname_mapper)

In [27]:
# reset the year column
df_pvs_pf_agg['Year'] = df_pvs_pf_agg['Date_cln'].dt.year

# rename date column
df_pvs_pf_agg = df_pvs_pf_agg.rename(columns={'Date_cln': 'Date'})

### Create long-form version of data

In [28]:
# convert to long format to be able to plot with hue as the FG
df_pvs_pf_agg_long = pd.wide_to_long(df_pvs_pf_agg.reset_index(), 
                                     stubnames=['hits', 'weight'], 
                                     i=['Site', 'Plot Name', 'Date'], 
                                     j='FG', 
                                     sep="_", 
                                     suffix=r'\w+').reset_index()

### Check data and save to disk

In [29]:
# plot linear fits for each functional group (FG)
sns.lmplot(x='hits',
           y='weight',
           col='FG',
           col_wrap=3,
           hue='FG',
           data=df_pvs_pf_agg_long,
           scatter=True, height=3.5, aspect=1.1)

In [30]:
# explore outliers
df_pvs_pf_agg[(df_pvs_pf_agg.hits_C3 < 20) & (df_pvs_pf_agg.weight_C3 > 100)]

Site  Plot Name       Date  Total Hits  hits_C4  hits_C3  hits_AnnGrass  \
2  clayey          1 2019-07-23          25        2       11              6   

   hits_Forb  hits_Brome  hits_VUOC  ...  hits_Total  weight_Total  weight_C4  \
2          6           2          4  ...          25         240.9       16.8   

   weight_C3  weight_Forb  weight_Brome  weight_VUOC  weight_AnnGrass  \
2      155.4         40.9           9.1         13.5             22.6   

   weight_SD  Year  
2        5.2  2019  

[1 rows x 21 columns]

In [31]:
# drop meaningless columns
df_pvs_pf_agg_long = df_pvs_pf_agg_long.drop(columns=['index'])
# save datasets to disk
df_pvs_pf_agg.to_csv('data/TB_pvsage_pf.csv', index=False)
df_pvs_pf_agg_long.to_csv('data/TB_pvsage_pf_long.csv', index=False)

# Questions:
* Are we ok with removing 2017 loamy sites altogether since SD is missing?
* Ask Katie
  * about Sub-shrub weights [YUGL, AGGL, ARFR, ARPE, GUSA, OOMU]